## R-alpha

In [2]:
!pip install smartapi-python
!pip install websocket-client
!pip install logzero
!pip install pyotp

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 KB 15.8 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 KB 2.7 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
from connect_api import connect_api, create_api_session
from data import get_data_stream, token_lookup, get_instrument_list
from strategy import market_pressure

In [2]:
api_key = '2VTNvx7z'
username = 'R258447'
pwd = '1499'
token_id = '3URP5C2ZUQSYTSEFELMM2CFJGM'

In [3]:
smartApi = connect_api(api_key)
data = create_api_session(token_id, username, pwd, smartApi)

[I 240423 09:36:11 smartConnect:121] in pool


In [4]:
token_list = [
        {
            "exchangeType": 1,
            "tokens": ["26009"]
        }
    ]

In [5]:
sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list)

### Order Book Strategy

In [ ]:
instrument_list = get_instrument_list()

tickers = ["WIPRO","ULTRACEMCO","UPL","TITAN","TECHM","TATASTEEL","TATAMOTORS",
           "TATACONSUM","TCS","SUNPHARMA","SBIN","SBILIFE","RELIANCE","POWERGRID",
           "ONGC","NESTLEIND","NTPC","MARUTI","M&M","LT","KOTAKBANK","JSWSTEEL",
           "INFY","INDUSINDBK","ITC","ICICIBANK","HDFC","HINDUNILVR","HINDALCO",
           "HEROMOTOCO","HDFCLIFE","HDFCBANK","HCLTECH","GRASIM","EICHERMOT",
           "DRREDDY","DIVISLAB","COALINDIA","CIPLA","BRITANNIA","BHARTIARTL",
           "BPCL","BAJAJFINSV","BAJFINANCE","BAJAJ-AUTO","AXISBANK","ASIANPAINT",
           "APOLLOHOSP","ADANIPORTS","ADANIENT"]


bid_ask_ratio = {}
long_trades = {}
short_trades = {}

for ticker in tickers:
    bid_ask_ratio[ticker] = []
    long_trades[ticker] = []
    short_trades[ticker] = []

pos_size = 10000

long_pos = []
short_pos = []

correlation_id = "stream_1" #any string value which will help identify the specific streaming in case of concurrent streaming
action = 1 #1 subscribe, 0 unsubscribe
mode = 3 #1 for LTP, 2 for Quote and 2 for SnapQuote
token_list = [{"exchangeType": 1, "tokens": [token_lookup(i,instrument_list) for i in tickers]}]


def on_data(ws, message):
    market_pressure(message)
    
def on_open(ws):
    print("on open")
    sws.subscribe(correlation_id, mode, token_list)
    
def on_error(ws, error):
    print(error)
    

# Assign the callbacks.
sws.on_open = on_open
sws.on_data = on_data
sws.on_error = on_error

sws.connect()